# Make soybean_vmd_weekly.csv

In [1]:
#VMD instalation
!pip install vmdpy

# Imports
import pandas as pd
import numpy as np
from vmdpy import VMD

# Pull soybean data
data_url = 'https://raw.githubusercontent.com/ptraver/data/main/bean_daily_V2.csv'
soybean_df = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='date')
soybean_df.head()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,value
date,
1972-01-03,306.25
1972-01-04,304.63
1972-01-05,308.38
1972-01-06,309.00
1972-01-07,309.38


In [2]:

# fix bad name
soybean_df.columns = ['value']

# take only 1974 - 2017
soybean_df = soybean_df['1974-01-01':'2017-12-31']


In [3]:
# downsample to weekly
soybean_df = soybean_df.resample('W').mean()
soybean_df.head()

,value
date,
1974-01-06,594.876667
1974-01-13,615.900000
1974-01-20,618.800000
1974-01-27,635.500000
1974-02-03,637.400000


In [4]:
# Make VMD

#. some sample parameters for VMD  
alpha = 2000       # moderate bandwidth constraint  
tau = 0.           # noise-tolerance (no strict fidelity enforcement)  
K = 7              # 7 modes -- same amount as emd on this set
DC = 0             # no DC part imposed  
init = 0           # initialize omegas uniformly  
tol = 1e-3

u = VMD(soybean_df['value'].values, alpha, tau, K, DC, init, tol)

# sum up all modes except highest frequency
v = np.zeros((len(u[0][0])))
for i in range(K - 1):
  v += u[0][i]

# Make a soybean_vmd dataframe
soybean_vmd = pd.DataFrame(v, columns=['value'], index=soybean_df.index[:-1])

In [5]:
# Save dataframe
soybean_vmd.to_csv('soybean_vmd_weekly.csv')

# End